In [1]:
import polars as pl

In [8]:
#scan parquet them convert timestamp column in to a Datetime format
df = (
    pl.scan_parquet("new_gps.parquet")
    .with_columns(pl.from_epoch(pl.col("timestamp"), "ms"))
    # truncate it by the hour
    .with_columns(pl.col("timestamp").dt.truncate("1h"))
    # Then group_by by the hour and calculate the mean, the max and min of the speed column
    .group_by(pl.col("timestamp"))
    .agg(
        pl.col("speed").mean().alias("mean_speed"),
        pl.col("speed").min().alias("min_speed"),
        pl.col("speed").max().alias("max_speed"),
    )
    # Filter for hour > 4
    .filter(pl.col("timestamp") > 4)
)

In [9]:
print(df.explain())

AGGREGATE
	[col("speed").mean().alias("mean_speed"), col("speed").min().alias("min_speed"), col("speed").max().alias("max_speed")] BY [col("timestamp")] FROM
  FILTER [(col("timestamp").cast(Int64)) > (4)] FROM

   WITH_COLUMNS:
   [col("timestamp").dt.truncate([String(1h)])]
     WITH_COLUMNS:
     [col("timestamp").strict_cast(Datetime(Milliseconds, None))]

        Parquet SCAN new_gps.parquet
        PROJECT 2/23 COLUMNS


In [11]:
lazy = (
    (pl.scan_parquet("new_gps.parquet"))
    .with_columns(pl.col("speed") * 3.6)
    .filter(pl.col("speed") > 10)
    .filter(pl.col("user_id") > 2)
)
print(lazy.explain())

FILTER [(col("speed")) > (10.0)] FROM
 WITH_COLUMNS:
 [[(col("speed")) * (3.6)]]

    Parquet SCAN new_gps.parquet
    PROJECT */23 COLUMNS
    SELECTION: [(col("user_id")) > (2)]
